In [89]:
import plotly.graph_objects as go
from pymongo import MongoClient, DESCENDING, ASCENDING
import pandas as pd
import numpy as np
import ta
import yfinance as yf
import logging

### Add New Time Interval

In [90]:
def fetch_data(symbol):
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    db = client['historic_data']
    collection = db['daily_stock_price']
    # Get one symbol data from the collection
    df = pd.DataFrame(list(collection.find({'symbol':symbol},{'_id': 0}).sort('date', ASCENDING)))
    return df

In [91]:
new_interval_record = {}

def create_new_interval_data(df, interval):
    new_df = df.set_index('date')
    new_df = new_df.groupby(pd.Grouper(freq=interval)).agg({'open': 'first',
                                                'high': 'max', 
                                                'low': 'min', 
                                                'close': 'last', 
                                                'volume': 'sum'})
    new_df.reset_index(inplace=True)
    new_df = new_df.dropna()
    return new_df

create_new_interval_data(df,'1D')

,date,open,high,low,close,volume
0,1999-01-22,0.040125,0.044783,0.035587,0.037618,2714688000
3,1999-01-25,0.040603,0.042036,0.037618,0.041559,510480000
4,1999-01-26,0.042036,0.042872,0.037737,0.038334,343200000
5,1999-01-27,0.038453,0.039409,0.036303,0.038215,244368000
6,1999-01-28,0.038215,0.038453,0.037856,0.038095,227520000
...,...,...,...,...,...,...
9380,2024-09-27,123.970001,124.029999,119.260002,121.400002,271009200
9383,2024-09-30,118.309998,121.500000,118.150002,121.440002,226553700
9384,2024-10-01,121.769997,122.440002,115.790001,117.000000,302094500
9385,2024-10-02,116.440002,119.379997,115.139999,118.849998,221845900


In [92]:
# Add bull and bear features in the dataframe
class add_features:
    def __init__(self, df):
        self.df = df.copy()

    def add_candlestick(self):
        self.df["BodyDiff"] = abs(self.df["open"] - self.df["close"])
        self.df["CandleStickType"] = np.where(self.df["open"] < self.df["close"], "green", "red")
        return self.df

    def continuous_increase(self, windows=3):
        for i in range(1, windows + 1):
            self.df[f"close_t-{i}"] = self.df["close"].shift(i)

        self.df['Incremental_High'] = (self.df['close'] > self.df['close_t-1']) \
                                        & (self.df['close_t-1'] > self.df['close_t-2']) \
                                        & (self.df['close_t-2'] > self.df['close_t-3'])
                                        
        self.df = self.df.drop(columns=[f"close_t-{i}" for i in range(1, windows + 1)])
        return self.df

    def macd_golden_cross(self):
        self.df['MACD_GOLDEN_CROSS'] = (self.df['MACD'] > self.df['MACD_SIGNAL']) & (self.df['MACD'] < 0)
        return self.df
    
    # Add Technical Indicators
    def add_technical(self):
        
        # Add ema dual channels technical indicators
        self.df['8EMA'] = ta.trend.ema_indicator(self.df['close'], window=8)
        self.df['13EMA'] = ta.trend.ema_indicator(self.df['close'], window=13)
        self.df['144EMA'] = ta.trend.ema_indicator(self.df['close'], window=144)
        self.df['169EMA'] = ta.trend.ema_indicator(self.df['close'], window=169)

        # Add MACD technical indicator
        self.df['MACD'] = ta.trend.macd(self.df['close'], 
                                        window_slow=26, 
                                        window_fast=12)
        
        self.df['MACD_SIGNAL'] = ta.trend.macd_signal(self.df['close'], 
                                                    window_slow=26, 
                                                    window_fast=12, 
                                                    window_sign=9)
        
        self.df['MACD_HIST'] = ta.trend.macd_diff(self.df['close'], 
                                                window_slow=26, 
                                                window_fast=12, 
                                                window_sign=9)
        
        # Add ATR technical indicator
        self.df["atr"] = ta.volatility.AverageTrueRange(high=self.df.high, 
                                                        low=self.df.low, 
                                                        close=self.df.close).average_true_range()
        
        self.df["atr"] = self.df.atr.rolling(window=30).mean()
        
        return self.df
    
    def apply(self):
        self.add_technical()
        self.add_candlestick()
        self.continuous_increase()
        self.macd_golden_cross()
        self.add_ema_band()
        
        # Use last 1000 rows of the dataframe
        self.df = self.df.tail(3000) if len(self.df) > 3000 else self.df
        return self.df


In [93]:
class add_alert:
    
    def __init__(self, df):
        self.df = df.copy()
        self.df['MACD_Alert'] = -1  
        self.df['Engulf_Alert'] = -1 
        self.df['dual_channel_Alert'] = -1  
        self.df['382_Alert'] = -1  
        self.window = 3  # Number of days to compare the stock price
        
    def engulf_alert(self):
        
        # Previous Candle
        prev_open = self.df['open'].shift(1)
        prev_close = self.df['close'].shift(1)
        
        # Current Candle
        current_open = self.df['open']
        current_close = self.df['close']

        # Bullish Engulfing: current green candle engulfs previous red candle
        bullish_engulfing = (
            (prev_close < prev_open) &  # Previous candle was red
            (current_close > current_open) &  # Current candle is green
            (current_open < prev_close) & 
            (current_close > prev_open)
        )

        # Bearish Engulfing: current red candle engulfs previous green candle
        bearish_engulfing = (
            (prev_close > prev_open) &  # Previous candle was green
            (current_close < current_open) &  # Current candle is red
            (current_open > prev_close) & 
            (current_close < prev_open)
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_engulfing, 'Engulf_Alert'] = 1
        self.df.loc[bearish_engulfing, 'Engulf_Alert'] = 0
        
        return self.df

    def macd_alert(self):
        
        # Pre-compute conditions
        macd_above_signal = self.df['MACD'] > self.df['MACD_SIGNAL']
        macd_increasing = self.df['MACD'].diff() > 0
        macd_below_zero = self.df['MACD'] >= 0

        # Bullish condition
        bullish_macd = (
            macd_above_signal &
            macd_increasing&
            macd_below_zero
        )
        
        # Bearish condition
        bearish_macd = (
            (self.df['MACD'] < self.df['MACD_SIGNAL']) &
            (self.df['MACD'] > self.df['MACD'].shift(-1))  
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_macd, 'MACD_Alert'] = 1
        self.df.loc[bearish_macd, 'MACD_Alert'] = 0

        return self.df

    def dual_channel(self):
        
        # Confirm Technical Indicators Conditions
        ema_8_gt_ema_13 = self.df['8EMA'] > self.df['13EMA']
        ema_13_gt_ema_169 = self.df['13EMA'] > self.df['169EMA']
        ema_8_gt_ema_144 = self.df['8EMA'] > self.df['144EMA']
        
        # Confirm Close price action
        close_ge_ema_13 = self.df['close'] >= self.df['13EMA']
        close_ge_ema_144 = self.df['close'] >= self.df['144EMA']
        
        # Confirm Candle fully above 13 EMA
        low_ge_ema_13 = self.df['low'] >= self.df['13EMA'] 
        green_candle = self.df['CandleStickType'] == 'green'    
        
        # Confirm Candle in selected range
        open_in_slow_ema = self.df['open'].between(self.df['169EMA_Lower'], self.df['169EMA_Upper'])
        
        # Confirm Candle type
        green_candle = self.df['CandleStickType'] == 'green'
        
        # Bullish Conditions
        
        bullish_scenario_1 = (
            ema_8_gt_ema_13 & 
            ema_13_gt_ema_169 & 
            close_ge_ema_13 &
            low_ge_ema_13 &
            green_candle
        )
        
        # Bearish Condition
        bearish_scenario_1 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['13EMA'] < self.df['8EMA'])
        )
        # Bullish Conditions 2
        bullish_scenario_2 = ( 
            ema_8_gt_ema_144 &
            ema_13_gt_ema_169 &
            close_ge_ema_144 &
            open_in_slow_ema &
            green_candle
            )
        
        # Bearish Condition 2
        bearish_scenario_2 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['8EMA'] < self.df['13EMA']) &
            ~ open_in_slow_ema
        )

        # Apply conditions on different price scenarios
        
        self.df.loc[bullish_scenario_1 | bullish_scenario_2, 'dual_channel_Alert'] = 1
        self.df.loc[bearish_scenario_1 | bearish_scenario_2, 'dual_channel_Alert'] = 0
        
        return self.df

    def bullish_382_alert(self):
        # Calculate Fibonacci 38.2% retracement level
        diff = self.df['high'] - self.df['low']
        fib_382_level = self.df['high'] - 0.382 * diff
        open_above_fib_382 = self.df['open'] > fib_382_level
        
        # Apply condition
        self.df.loc[open_above_fib_382, '382_Alert'] = 1
        self.df.loc[~open_above_fib_382, '382_Alert'] = 0
        
        return self.df

    def apply(self):
        self.engulf_alert()
        self.macd_alert()
        self.dual_channel()
        self.bullish_382_alert()
        return self.df

In [94]:
def insert_technical_data(db, tech_collection_name, symbol, df, interval, current_date):
    # Check for last record in the collection
    last_record = db[tech_collection_name].find_one({"symbol": symbol, "interval": interval},
                                                    sort=[('timestamp', DESCENDING)])
    if last_record:
        last_date_in_db = pd.to_datetime(last_record['date']).strftime('%Y-%m-%d')
        if last_date_in_db == current_date:
            logging.info(f"Data for {symbol} is up to date")
            return
        new_records_df = df[df['date'] > last_date_in_db]
    else:
        new_records_df = df

    if not new_records_df.empty:
        new_records_df["symbol"] = symbol
        new_records_df["interval"] = interval
        new_records_df["timestamp"] = pd.to_datetime(new_records_df["date"])
        db[tech_collection_name].insert_many(new_records_df.to_dict(orient='records'))
        logging.info(f"Inserted {len(new_records_df)} records into {tech_collection_name} collection")


In [118]:
def process_made(db, tech_collection_name, new_intervals, current_date):
    distinct_symbols = db[tech_collection_name].distinct('symbol')
    for symbol in distinct_symbols:
        df = fetch_data(symbol)       
        for interval in new_intervals:
            logging.info(f"Adding new interval {interval} for symbol {symbol}...")
            
            new_df = create_new_interval_data(df, interval)
            
            logging.info(f"Processing completed for {symbol} in {interval} interval")
            
            # Check if the data is already in the collection
            last_record = db[tech_collection_name].find_one({"symbol": symbol, "interval": interval},
                                                            sort=[('timestamp', DESCENDING)])
            current_date_dt = pd.to_datetime(current_date)
            if last_record:
                # Ensure the date is in pandas datetime format
                last_date_in_db = pd.to_datetime(last_record['timestamp']).strftime('%Y-%m-%d')
                
                # Check if the data is up to date
                if last_date_in_db == current_date_dt:
                    logging.info(f"The made {interval} Data for {symbol} is up to date")
                    continue
                else:
                    # Take only the fetched record is newer
                    new_records_df = new_df[new_df['date'] > last_date_in_db]
            # If the symbol does not exist in the technical collection
            else:
                new_records_df = new_df
            # Insert the new interval data accordingly
            if not new_records_df.empty:
                insert_technical_data(db, tech_collection_name, symbol, new_records_df, interval,current_date_dt)
                logging.info(f"Inserted new interval data for {symbol} in {interval} interval")
                
process_made(MongoClient('localhost')['historic_data'], 'processed_stock_data', ['2D','3D','4D','5D'], '2024-10-11')

### Defien the Stock Price Movement Velocity

In [119]:
def fetch_and_prepare_data(symbol):
    """
    Fetch and prepare data for a given stock symbol.
    
    Parameters:
    symbol (str): The stock symbol to fetch data for.
    
    Returns:
    pd.DataFrame: A DataFrame containing the prepared data.
    """
    # Fetch all interval data for the selected stock
    lst = list(MongoClient('localhost')['historic_data']['processed_stock_data']\
        .find({'symbol': symbol},{'_id': 0, 'close': 1, 'date': 1, "interval": 1, "13EMA": 1,"169EMA": 1, '169EMA_Lower':1, '169EMA_Upper':1})\
            .sort([('interval', DESCENDING), ('date', ASCENDING)]))
    
    # Convert the list to a dataframe
    df = pd.DataFrame(lst)

    # Replace daily to 1D
    df['interval'] = df['interval'].replace('daily', '1D')

    # Drop weekly data
    df = df[df['interval'] != 'weekly']

    # Drop rows with missing values
    df = df.dropna()

    return df


#### Support Loss Function

Let:
- $C_i$ be the closing price at time step $i$
- ${EMA}_{13,i}$ be the 13-period Exponential Moving Average (EMA) at time step $i$
- $N$ be the total number of time steps in the interval

The **support loss** is calculated as the sum of penalties for all time steps where the closing price \( C_i \) is below the 13 EMA:

$$
\text{Support Loss} = \sum_{i=1}^{N} \max(0,  \text{EMA}_{13,i} - C_i)
$$

Where:
- If $C_i \geq \text{EMA}_{13, i}$, the loss for that time step is 0 (no penalty).
- If $C_i < \text{EMA}_{13, i}$, the loss for that time step is the absolute difference ${EMA}_{13,i} - C_i$.

In [129]:
class SupportLossCalculator:
    def __init__(self, data):
        self.data = data

    def support_loss(self, closing_prices, ema_13):
        """
        Custom loss function to find intervals where the 13 EMA acts as support.
        Penalizes any instance where the closing price falls below the 13 EMA.
        """
        # Calculate the difference between the closing price and the 13 EMA
        differences = closing_prices - ema_13

        # Penalize cases where the closing price is below the 13 EMA (negative values)
        penalties = np.where(differences < 0, np.abs(differences), 0)

        # Sum up the penalties to form the loss
        loss = np.sum(penalties)

        return loss

    def find_best_interval(self):
        """
        Find the interval with the lowest time-weighted 'price velocity', 
        i.e., the one where the closing price fits the 13 EMA line the best.
        """
        intervals = self.data['interval'].unique()
        best_interval = None
        min_loss = float('inf')

        for interval in intervals:
            if 'D' not in interval:
                continue  # Skip non-numeric intervals
            interval_data = self.data[self.data['interval'] == interval].iloc[-60:]
            closing_prices = interval_data['close'].values
            ema_13 = interval_data['13EMA'].values

            # Compute time-weighted loss for this interval
            loss = self.support_loss(closing_prices, ema_13)

            if loss < min_loss:
                min_loss = loss
                best_interval = interval

        return best_interval, min_loss

# Usage
data = fetch_and_prepare_data('PLTR')
calculator = SupportLossCalculator(data)
best_interval, min_loss = calculator.find_best_interval()
best_interval, min_loss

('2D', 8.302519059497886)

### Predefined Velocity Support Testing Alert

In [114]:
def effective_velocity_support_alert(df, interval, recovery_days=3, epilson=0.05):
    """
    Count how many times the closing price touches the 169 EMA band and check if it quickly recovers.
    
    Parameters:
    df (DataFrame): DataFrame containing 'close', '169EMA' columns.
    interval (str): The interval to analyze.
    recovery_days (int): The number of days within which the stock must recover above the band.
    
    Returns:
    recovery_count (int): Number of times the price quickly recovers above the band after touching it.
    """
    
    # Detect if the price touches the band
    df = df.copy()
    df = df[df['interval'] == interval]
    
    df['13_ema_upper'] = df['13EMA'] * (1 + epilson)
    df['13_ema_lower'] = df['13EMA'] * (1 - epilson)
    
    df['above_13_emas'] = df['close'] > df['13EMA']
    df['below_13_emas'] = df['close'] < df['13EMA']
    df['inside_13_ema_band'] = (df['close'] > df['13_ema_lower']) & (df['close'] < df['13_ema_upper'])
    df['vel_recovery'] = False
    df['vel_recover_failed'] = False
    
    for i in df.index:
        
        if i + recovery_days < df.index[-1]:
            
            if df.loc[i+recovery_days, 'inside_13_ema_band'] and df.loc[i-recovery_days:i, 'above_13_emas'].any():
                # Check if the price moves above the upper bound within the next 'recovery_days'
                future_prices = df.loc[i:i+recovery_days, 'close']
                if (future_prices > df.loc[i, '13_ema_upper']).all():
                    df.loc[i, 'vel_recovery'] = True
                        
            if df.loc[i+recovery_days, 'inside_13_ema_band'] and df.loc[i-recovery_days:i, 'below_13_emas'].any():
                # Check if the price moves above the upper bound within the next 'recovery_days'
                    future_prices = df.loc[i:i+recovery_days, 'close']
                    if (future_prices < df.loc[i, '13EMA']).all():
                        df.loc[i, 'vel_recover_failed'] = True
                        
    df = df.drop(columns=['13_ema_upper', '13_ema_lower', 'above_13_emas', 'below_13_emas', 'inside_13_ema_band'])
    return df 

nvda_df = fetch_and_prepare_data('NVDA')
effective_velocity_support_alert(nvda_df,'1D')

,169EMA_Upper,169EMA,close,169EMA_Lower,date,13EMA,interval,vel_recovery,vel_recover_failed
756,18.839146,17.942043,20.706453,17.044941,2021-10-01,21.325370,1D,False,True
757,18.860839,17.962704,19.698181,17.064569,2021-10-04,21.092914,1D,False,True
758,18.891144,17.991566,20.415951,17.091987,2021-10-05,20.996205,1D,False,False
759,18.924163,18.023012,20.664524,17.121862,2021-10-06,20.948822,1D,False,False
760,18.961418,18.058493,21.038881,17.155568,2021-10-07,20.961688,1D,False,False
...,...,...,...,...,...,...,...,...,...
1507,106.268990,101.208562,121.400002,96.148134,2024-09-27,118.738305,1D,False,False
1508,106.518908,101.446579,121.440002,96.374250,2024-09-30,119.124262,1D,False,False
1509,106.711039,101.629561,117.000000,96.548083,2024-10-01,118.820796,1D,False,False
1510,106.923762,101.832154,118.849998,96.740546,2024-10-02,118.824968,1D,False,False


### 169EMA Support Testing Alert

- Add columns to detect whether the closing price enters the predefined 169 EMA band.
- Detect if the closing price is able to recover and move back above the 169 EMA after touching the band.

* 169 EMA Band Calculation
$$
{EMA_{169} Band} = EMA_{169} \times (1 \pm \epsilon)
$$
Where  $\epsilon$ is a small percentage to define the upper and lower bounds of the band.

* Recovery Indicator
Let ${Recovery}_{i}$ be an indicator where $i$ is either 0 or 1:
- $1$: The stock price recovered after touching the 169 EMA band and moved above the upper bound within the specified number of days.
- $0$: The stock price did not recover after touching the 169 EMA band.

In [117]:
def effective_ema_support_alert(df, interval, recovery_days=3, epilson=0.05):
    """
    Count how many times the closing price touches the 169 EMA band and check if it quickly recovers.
    
    Parameters:
    df (DataFrame): DataFrame containing 'close', '169EMA' columns.
    interval (str): The interval to analyze.
    recovery_days (int): The number of days within which the stock must recover above the band.
    
    Returns:
    recovery_count (int): Number of times the price quickly recovers above the band after touching it.
    """
    
    # Detect if the price touches the band
    df = df.copy()
    df = df[df['interval'] == interval]
    
    df['169EMA_Lower'] = df['169EMA'] * (1 - epilson)
    df['169EMA_Upper'] = df['169EMA'] * (1 + epilson)
    
    df['above_169_emas'] = df['close'] > df['169EMA']
    df['below_169_emas'] = df['close'] < df['169EMA']
    df['inside_169_ema_band'] = (df['close'] > df['169EMA_Lower']) & (df['close'] < df['169EMA_Upper'])
    df['recovery'] = False
    df['recover_failed'] = False
    
    for i in df.index:
        
        if i + recovery_days < df.index[-1]:
            
            if df.loc[i+recovery_days, 'inside_169_ema_band'] and df.loc[i-recovery_days:i, 'above_169_emas'].any():
                # Check if the price moves above the upper bound within the next 'recovery_days'
                future_prices = df.loc[i:i+recovery_days, 'close']
                if (future_prices > df.loc[i, '169EMA_Upper']).all():
                    df.loc[i, 'recovery'] = True
                        
            if df.loc[i+recovery_days, 'inside_169_ema_band'] and df.loc[i-recovery_days:i, 'below_169_emas'].any():
                # Check if the price moves above the upper bound within the next 'recovery_days'
                    future_prices = df.loc[i:i+recovery_days, 'close']
                    if (future_prices < df.loc[i, '169EMA']).all():
                        df.loc[i, 'recover_failed'] = True
                        
    df = df.drop(columns=['above_169_emas', 'below_169_emas', 'inside_169_ema_band', '169EMA_Lower', '169EMA_Upper'])
    return df 

nvda_df = fetch_and_prepare_data('NVDA')
effective_ema_support_alert(nvda_df,'1D')

,169EMA,close,date,13EMA,interval,recovery,recover_failed
756,17.942043,20.706453,2021-10-01,21.325370,1D,False,False
757,17.962704,19.698181,2021-10-04,21.092914,1D,False,False
758,17.991566,20.415951,2021-10-05,20.996205,1D,False,False
759,18.023012,20.664524,2021-10-06,20.948822,1D,False,False
760,18.058493,21.038881,2021-10-07,20.961688,1D,False,False
...,...,...,...,...,...,...,...
1507,101.208562,121.400002,2024-09-27,118.738305,1D,False,False
1508,101.446579,121.440002,2024-09-30,119.124262,1D,False,False
1509,101.629561,117.000000,2024-10-01,118.820796,1D,False,False
1510,101.832154,118.849998,2024-10-02,118.824968,1D,False,False


### Anticipated Support driven by Velocity Movement

In [ ]:
# Fetch and prepare data for the selected stock
symbol = 'NVDA'
df = fetch_and_prepare_data(symbol)

# Compute the expected support alert
def anticipated_support(df):
    # If the price drops below the 13 EMA and cannot recover within 3 days
    for date in df.index:
        if df.loc[date,'close'] < df.loc[date, '13EMA'] and df.loc[date, 'recover_failed']:
            df.loc[date, 'anticipated_support'] = df.loc[date,'169EMA']

